In [1]:
import psycopg2
import json
from datetime import datetime as dt
import pandas as pd
import re 
from textblob import TextBlob 

In [2]:
p="dataisgreat"

In [3]:
conn = psycopg2.connect(user = "postgres",
                                  password = f"{p}",
                                  host = "127.0.0.1",
                                  port = "3306",
                                  database = "postgres")

In [4]:
cursor = conn.cursor()

In [5]:
tweet_list = []
followers = []
cursor.execute("select * from tweets")
tweets = cursor.fetchall()
if not tweets:
    print("empty")
for row in tweets:
    for col in row:
        if type(col) is dict:
            tweet_list.append(col['text'])
            followers.append(col['user']['followers_count'])

In [6]:
remove_list = [r'@[A-Za-z0-9]+','https?://[A-Za-z0-9./]+','\n','RT :',r'_[A-Za-z0-9]+','  ']

In [7]:
def clean_tweets(remove_list,tweet_list):
    for item in remove_list:
        tweet_list = [re.sub(item,'',tweet) for tweet in tweet_list]
    return tweet_list

In [9]:
cleaned = clean_tweets(remove_list,tweet_list)
cleaned[:5]

['There are so many hilarious comments in You Tubechat on @ show tonight.',
 ' Democrat frontrunner Michael Bloomberg says he wants to make the whole country "behave" while dressed as Mary Poppins.Wel…',
 'The latest COACH!#election2020 #misleadingtweets',
 'THERE IS NO ONE TO VOTE FOR! Centrists demand Valium to get them through #Election2020',
 'RT : Calling all voters!! Are you ready for November?👉🏿RT to spread the word. 🗳️ #Election2020 http…']

In [146]:
# tweet_text = pd.DataFrame(tweet)
# tweet_text.rename(columns = {0: "tweets"}, inplace = True)
# tweet_text.head()

In [11]:
sentiment_objects = []
sentiments = []
for each in cleaned:
    sentiments.append(TextBlob(each).polarity)

    
# sentiment_objects.polarity, sentiment_objects
# print(sentiment_objects[0])
    
print(sentiments[:10])

[0.5, 0.2, 0.5, 0.0, 0.2, 0.0, 0.0, 0.4, 0.5, 0.0]


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
)
features = vectorizer.fit_transform(
    cleaned
)
features_nd = features.toarray() # for easy usage

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        cleaned, 
        sentiments,
        train_size=0.80, 
        random_state=42)

In [29]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [30]:
model.fit(X_train,y_train)

ValueError: Expected 2D array, got 1D array instead:
array=[" Don't take your vote with you!Check your vote and put your vote in the ballot box!You can be a Texas voter with…"
 ' Ok Christians!We are not ashamed of the gospel and boldly proclaim our Lord and Savior Jesus.Let’s flip this poll.Ple…'
 "Let's go Ohio! // I’m attending Pete for America’s event, “Debate &amp; Post Card Party in Cleveland!” – sign up now to… "
 ... 'Translation:#NYTimes wants #Buttigieg in #Election2020 '
 ' The truth that #msnbc n #CNN won’t show you!#socialism #socialismsucks #berniesanders #bernie2020 #bernie #medicaid #medica…'
 ' we live in a country that seems to care more about cheating in baseball than cheating in elections. #America #Election2020'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [20]:
print(f"Training score: {model.score(X_train,y_train)}")
print(f"Testing score: {model.score(X_test,y_test)}")

Training score: 0.9999921190032599
Testing score: -1.9049635333050607e+20


In [31]:
print(X_test,y_test[0])

[' Floridais requiring that all persons seeking employment in the State be confirmed e-verified legal resi…', 'This is legitimately what a Russian oligarch does. #BloombergIsAnOligarch #Election2020 #PsyOps', ' So the #DNC knows they have a losing ticket. Time to bring out Russia, Russia, Russia?? #FridayThoughts #Election2020 #VoteRe…', ' Wait!Russian collusion, oh my. Wrong guy! This is not how it was scripted by the DNC! Gee, why would Russia want Bernie…', "RT : Trump Rallygoers in Phoenix: 18 Percent Were Democrats and About a Quarter Didn't Vote in 2016 …", ' Do you believe America would be better off if more Christians served in elected office?#Retweet #Poll #Election2020 #…', ' Looks like...less than 3 weeks!👌🏻🤣👍🏻 Never stop being you, ...with each shovel load of #RussianCollusion you heap…', ' Attn #California!NPP ballots are being sent to registered Dems. DO NOT CAST THAT BALLOT. There are no presidential candid…', 'The stakes in #Election2020 are escalating every minute trum

In [14]:
emolex_df.emotion.unique()

array(['joy', 'negative', 'positive', 'sadness', 'surprise', 'trust',
       'anger', 'anticipation', 'disgust', 'fear'], dtype=object)

In [35]:
# NRC
filepath = "NRC-Sentiment-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], skiprows=45, sep='\t')

In [26]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import os

In [32]:
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

ValueError: Expected 2D array, got 1D array instead:
array=[" Don't take your vote with you!Check your vote and put your vote in the ballot box!You can be a Texas voter with…"
 ' Ok Christians!We are not ashamed of the gospel and boldly proclaim our Lord and Savior Jesus.Let’s flip this poll.Ple…'
 "Let's go Ohio! // I’m attending Pete for America’s event, “Debate &amp; Post Card Party in Cleveland!” – sign up now to… "
 ... 'Translation:#NYTimes wants #Buttigieg in #Election2020 '
 ' The truth that #msnbc n #CNN won’t show you!#socialism #socialismsucks #berniesanders #bernie2020 #bernie #medicaid #medica…'
 ' we live in a country that seems to care more about cheating in baseball than cheating in elections. #America #Election2020'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.